<a href="https://colab.research.google.com/github/SeongjaeP/4_2_finalproject/blob/main/mimic_cxr_seongjae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
metadata = pd.read_csv('/content/drive/MyDrive/dive_medical/mimic-cxr-2.0.0-metadata.csv')
metadata

## 정면, PA 데이터 전처리

In [ ]:
pa_metadata = metadata[(metadata['ViewPosition'] == 'PA') & (metadata['PerformedProcedureStepDescription'] == 'CHEST (PA AND LAT)')]
pa_metadata

In [ ]:
cxr_data = pd.read_csv('/content/drive/MyDrive/dive_medical/mimic-cxr-2.0.0-chexpert.csv')
cxr_data

In [ ]:
merge_df = pd.merge(pa_metadata, cxr_data ,on='study_id', how='left')

merge_df.fillna(0, inplace = True)
merge_df

In [ ]:
import torch
from transformers import DistilGPT2LMHeadModel, GPT2Tokenizer

class ImageCaptioningModel:
    def __init__(self, model_name="distilgpt2"):
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = DistilGPT2LMHeadModel.from_pretrained(model_name)

    def combine_features(self, image_features, text_features):

        return text_features

    def generate_caption(self, combined_features):
        inputs = self.tokenizer.encode(combined_features, return_tensors="pt", add_special_tokens=True)
        outputs = self.model.generate(inputs, max_length=50, num_beams=5, early_stopping=True)
        caption = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return caption

    def forward(self, image_features, text_features):
        combined_features = self.combine_features(image_features, text_features)
        caption = self.generate_caption(combined_features)
        return caption

### study_id 리스트로 뽑기
토큰화해야할 txt들

In [ ]:
study_id_list = merge_df['study_id'].tolist()
study_id_list

In [ ]:
len(study_id_list)

In [ ]:
chest_df = merge_df

## 폐 리스트만 뽑기

In [ ]:
lung_conditions = ['Atelectasis', 'Consolidation', 'Edema', 'Lung Lesion', 'Lung Opacity', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'No Finding']
lung_df = chest_df[lung_conditions]
lung_df

In [ ]:
# 각 조건별 '1'의 개수 확인
condition_counts = lung_df[lung_df == 1].count()

# 겹쳐 있는 클래스의 개수 계산
# 각 행에서 '1'의 개수를 세고, 1보다 큰 경우만 필터링하여 총 개수 계산
overlapping_classes_count = (lung_df[lung_conditions] == 1).sum(axis=1)
overlapping_cases = overlapping_classes_count[overlapping_classes_count > 1].count()

print("각 조건별 '1'의 개수:")
print(condition_counts)
print("\n겹쳐 있는 클래스의 총 개수:", overlapping_cases)

## 상관관계 히트맵

안쓰는게 좋을듯

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 겹쳐있는 데이터 필터링
overlapping_df = lung_df[(lung_df[lung_conditions] == 1).sum(axis=1) > 1]

# 상관관계 계산
correlation_matrix = overlapping_df.corr()

# 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Overlapping Lung Conditions Correlation Heatmap')
plt.show()

## subject_id_x